In [1]:
import numpy as np
import math
from kapteyn import kmpfit

from astropy.table import Table, join
from astropy.io import fits
from astropy.convolution import convolve, Gaussian1DKernel

import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import rc
%matplotlib inline

import my_coadd_spectra
from desispec.coaddition import coadd_cameras
import desispec.io

from desitarget.targetmask import desi_mask
from desitarget.sv1 import sv1_targetmask
from desitarget.sv2 import sv2_targetmask
from desitarget.sv3 import sv3_targetmask
sv1_desi_mask = sv1_targetmask.desi_mask
sv2_desi_mask = sv2_targetmask.desi_mask
sv3_desi_mask = sv3_targetmask.desi_mask
import specprodDB.load as db
from desiutil.log import get_logger, DEBUG
from prospect.specutils import Spectra
from prospect import viewer

import time

import logging
logging.getLogger().setLevel(logging.WARNING)
from IPython.display import clear_output

import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import json
import pickle

plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (24,8)

In [2]:
with fits.open("/global/cfs/cdirs/desi/public/dr1/vac/dr1/fastspecfit/iron/v2.1/catalogs/fastspec-iron.fits", memmap = True) as hdul:
    ELG = hdul[1].data[((hdul[2].data["DESI_TARGET"] & desi_mask['ELG']) != 0)]

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def my_selection(save = None, ELG = ELG, redshift_range_OII = [0.1, 0.4], OII_SNR = 5, OIII_SNR = 5, HALPHA_SNR = 5, redshift_range_Lya = [1.96, 7.08], Lya_SNR = 5, CIV_SNR = 2, info_fits = False):
    
    def BOX_SNR(dataset, feature):
        signal = dataset[str(feature) + "_BOXFLUX"]
        noise = dataset[str(feature) + "_BOXFLUX_IVAR"]
        preSNR = signal / noise ** (-1 / 2)
        return np.nan_to_num(preSNR)
    
    mask_redshift_low = ELG["Z"] < redshift_range_Lya[1]
    mask_redshift_high = ELG["Z"] > redshift_range_Lya[0]
    selection = ELG[mask_redshift_low & mask_redshift_high] 
    ELG_noQSO = selection[BOX_SNR(selection, "CIV_1549") < CIV_SNR]
    ELG_noQSO = ELG_noQSO[BOX_SNR(ELG_noQSO, "LYALPHA") > Lya_SNR]
    selection1 = ELG_noQSO[ELG_noQSO["LYALPHA_NPIX"] < 50]

    mask_redshift_low = ELG["Z"] < redshift_range_OII[1]
    mask_redshift_high = ELG["Z"] > redshift_range_OII[0]
    selection2 = ELG[mask_redshift_low & mask_redshift_high] 

    mask_box_OII = BOX_SNR(selection2, "OII_3726") > OII_SNR
    selection2 = selection2[mask_box_OII]
    
    mask_box_OIII = BOX_SNR(selection2, "OIII_5007") < OIII_SNR
    selection2 = selection2[mask_box_OIII]
    
    mask_box_HALPHA = BOX_SNR(selection2, "HALPHA") < HALPHA_SNR
    selection2 = selection2[mask_box_HALPHA]
    
    c1 = fits.Column(name = 'TARGETID', array = np.concatenate((selection1["TARGETID"], selection2["TARGETID"])), format = 'K')
    c2 = fits.Column(name = 'HEALPIX', array = np.concatenate((selection1["HEALPIX"], selection2["HEALPIX"])), format = 'K')
    c3 = fits.Column(name = 'SURVEY', array = np.concatenate((selection1["SURVEY"], selection2["SURVEY"])), format = '20A')
    c4 = fits.Column(name = 'PROGRAM', array = np.concatenate((selection1["PROGRAM"], selection2["PROGRAM"])) , format = '20A')
    c5 = fits.Column(name = 'Z', array = np.concatenate((selection1["Z"], selection2["Z"])) , format = 'E')
    c6 = fits.Column(name = 'RCHI2', array = np.concatenate((selection1["RCHI2"], selection2["RCHI2"])) , format = 'E')

    table_hdu = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6])
    
    if info_fits == True:
        primary_hdu = fits.PrimaryHDU()
        hdul1 = fits.HDUList([primary_hdu, table_hdu])
        hdul1.writeto('../DESI_LAE_dataset/catalog_iron_info.fits', overwrite = True)
    else:
        pass
    
    spectra = []
    print("First stage finsishes!")
    print(f"{len(hdul1[1].data)} Found in first stage")
    print("Start second stage")
    
    n = 0
    
    for i in hdul1[1].data:
        n += 1
        print(f"\rProcess: {str(n / len(hdul1[1].data) * 100)} %", end = "")
        dic = {}
        path = f"/global/cfs/cdirs/desi/spectro/redux/iron/healpix/{i['SURVEY']}/{i['PROGRAM']}/{i['HEALPIX'] // 100}/{i['HEALPIX']}/coadd-{i['SURVEY']}-{i['PROGRAM']}-{i['HEALPIX']}.fits"
        try:
            with fits.open(path, memmap = True) as hdul2:
                targetid_file = hdul2[1].data["TARGETID"]
                index = np.argwhere(targetid_file == i["TARGETID"])
                if index.shape[0] * index.shape[1] == 1:
                    wave = np.concatenate((hdul2[3].data, hdul2[8].data, hdul2[13].data))
                    flux = np.concatenate((hdul2[4].data[index[0][0]], hdul2[9].data[index[0][0]], hdul2[14].data[index[0][0]]))
                    ivar = np.concatenate((hdul2[5].data[index[0][0]], hdul2[10].data[index[0][0]], hdul2[15].data[index[0][0]]))
                    bitmask = np.concatenate((hdul2[6].data[index[0][0]], hdul2[11].data[index[0][0]], hdul2[16].data[index[0][0]]))
                    # spectra_obj = Spectra(bands = ["b"], wave = {"b": hdul2[3].data}, flux = {"b": np.array([hdul2[4].data[index[0][0]]])}, ivar = {"b": np.array([hdul2[5].data[index[0][0]]])}, 
                    #                       mask = {"b": np.array([hdul2[6].data[index[0][0]]], dtype = np.int64)})
                    dic["TARGETID"] = i["TARGETID"]
                    dic["HEALPIX"] = i["HEALPIX"]
                    dic["RCHI2"] = i["RCHI2"]
                    dic["Z"] = i["Z"]
                    dic["WAVELENGTH"] = wave
                    dic["FLUX"] = flux
                    dic["IVAR"] = ivar
                    dic["NITMASK"] = bitmask
                    dic["_dr"] = "DESI-IRON"
                    spectra.append(dic)
                else:
                    print(f"\nMultiple targetid:{i['TARGETID']}")
        except:
            print(f"\nOther problems:{i['TARGETID']}")


    print("\nSecond stage finsishes!")
    
    if save != None:
        with open(str(save) + "iron_spectra_mask.pkl", "wb") as fh:
            pickle.dump(spectra, fh)
    else:
        pass
        
    return spectra

In [7]:
def desispec_selection(save = None, ELG = ELG, redshift_range_OII = [0.1, 0.4], OII_SNR = 5, OIII_SNR = 5, HALPHA_SNR = 5, redshift_range_Lya = [1.96, 7.08], Lya_SNR = 5, CIV_SNR = 2, info_fits = False):
    
    def BOX_SNR(dataset, feature):
        signal = dataset[str(feature) + "_BOXFLUX"]
        noise = dataset[str(feature) + "_BOXFLUX_IVAR"]
        preSNR = signal / noise ** (-1 / 2)
        return np.nan_to_num(preSNR)
    
    mask_redshift_low = ELG["Z"] < redshift_range_Lya[1]
    mask_redshift_high = ELG["Z"] > redshift_range_Lya[0]
    selection = ELG[mask_redshift_low & mask_redshift_high] 
    ELG_noQSO = selection[BOX_SNR(selection, "CIV_1549") < CIV_SNR]
    ELG_noQSO = ELG_noQSO[BOX_SNR(ELG_noQSO, "LYALPHA") > Lya_SNR]
    selection1 = ELG_noQSO[ELG_noQSO["LYALPHA_NPIX"] < 50]

    mask_redshift_low = ELG["Z"] < redshift_range_OII[1]
    mask_redshift_high = ELG["Z"] > redshift_range_OII[0]
    selection2 = ELG[mask_redshift_low & mask_redshift_high] 

    mask_box_OII = BOX_SNR(selection2, "OII_3726") > OII_SNR
    selection2 = selection2[mask_box_OII]
    
    mask_box_OIII = BOX_SNR(selection2, "OIII_5007") < OIII_SNR
    selection2 = selection2[mask_box_OIII]
    
    mask_box_HALPHA = BOX_SNR(selection2, "HALPHA") < HALPHA_SNR
    selection2 = selection2[mask_box_HALPHA]

    c1 = fits.Column(name = 'TARGETID', array = np.concatenate((selection1["TARGETID"], selection2["TARGETID"])), format = 'K')
    c2 = fits.Column(name = 'HEALPIX', array = np.concatenate((selection1["HEALPIX"], selection2["HEALPIX"])), format = 'K')
    c3 = fits.Column(name = 'SURVEY', array = np.concatenate((selection1["SURVEY"], selection2["SURVEY"])), format = '20A')
    c4 = fits.Column(name = 'PROGRAM', array = np.concatenate((selection1["PROGRAM"], selection2["PROGRAM"])) , format = '20A')
    c5 = fits.Column(name = 'Z', array = np.concatenate((selection1["Z"], selection2["Z"])) , format = 'E')
    c6 = fits.Column(name = 'RCHI2', array = np.concatenate((selection1["RCHI2"], selection2["RCHI2"])) , format = 'E')

    table_hdu = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6])
    
    if info_fits == True:
        primary_hdu = fits.PrimaryHDU()
        hdul1 = fits.HDUList([primary_hdu, table_hdu])
        hdul1.writeto('../DESI_LAE_dataset/catalog_iron_info.fits', overwrite = True)
    else:
        pass
    
    spectra = []
    print("First stage finsishes!")
    print(f"{len(table_hdu.data)} Found in first stage")
    print("====================================================================================")
    print("Start second stage")
    
    n = 0
    
    for i in table_hdu.data:
        n += 1
        print(f"\rProcess: {str(n / len(table_hdu.data) * 100)} %", end = "")
        dic = {}
        path = f"/global/cfs/cdirs/desi/spectro/redux/iron/healpix/{i['SURVEY']}/{i['PROGRAM']}/{i['HEALPIX'] // 100}/{i['HEALPIX']}/coadd-{i['SURVEY']}-{i['PROGRAM']}-{i['HEALPIX']}.fits"
        coadd = desispec.io.read_spectra(path, targetids = i["TARGETID"], skip_hdus = ["EXP_FIBERMAP", "SCORES", "EXTRA_CATALOG"])
        spectra_obj = coadd_cameras(coadd)
        dic["spectrum"] = spectra_obj
        dic["TARGETID"] = i["TARGETID"]
        dic["HEALPIX"] = i["HEALPIX"]
        dic["RCHI2"] = i["RCHI2"]
        dic["Z"] = i["Z"]
        dic["_dr"] = "DESI-IRON"
        spectra.append(dic)
        clear_output()

    print("\nSecond stage finsishes!")
    
    if save != None:
        with open(str(save) + "iron_spectra_obj.pkl", "wb") as fh:
            pickle.dump(spectra, fh)
    else:
        pass
        
    return spectra

In [ ]:
spectra = my_selection(save = "../DESI_LAE_dataset/")

In [ ]:
spectra = desispec_selection(save = "../DESI_LAE_dataset/")

In [ ]:
a = viewer.plotspectra(spectra = spectra_obj, html_dir = "../DESI_LAE_dataset/")

In [ ]:
def plot_spec(spectra, end, init = 0, path = None, Lya = True):
    for i in range(init, end):
        record = spectra[i]
    
        flux = record["flux"]
        wavelength = record["wavelength"]
        redshift = record["redshift"]
        ivar = record["ivar"]
        targetid = record["specid"]

        w = wavelength
        f = flux
        iv = np.sqrt(ivar) ** -1

        if redshift > 0.4:
            continue
        if np.max(f) == np.inf:
            f[np.where(f == np.inf)] = np.median(f)
        elif np.max(iv) == np.inf:
            iv[np.where(iv == np.inf)] = np.median(iv)
        
        if Lya == True:
            min = 3726.032 * (redshift + 1) - 25
            max = 3726.032 * (redshift + 1) + 25
            mask_min = wavelength > min
            mask_max = wavelength < max
            w = wavelength[mask_min & mask_max]
            f = flux[mask_min & mask_max]
            iv = np.sqrt(ivar[mask_min & mask_max]) ** -1
            
            if redshift > 0.4:
                continue
            if np.max(f) == np.inf:
                f[np.where(f == np.inf)] = np.median(f)
            elif np.max(iv) == np.inf:
                iv[np.where(iv == np.inf)] = np.median(iv)
        else:
            pass

        plt.close()
        plt.title(f"target ID = {targetid}\n"
                  f"Redshift = {redshift}\n", loc = 'left')
        plt.xlabel('$\lambda\ [\AA]$')
        plt.ylabel('$f_{\lambda}$ $(10^{-17}$ $erg$ $s^{-1}$ $cm^{-2}$ $\AA^{-1})$')
        
        # Plot unsmoothed spectrum in grey
        plt.plot(w, f, color = 'k', alpha = 0.2, label = 'Unsmoothed spectrum')
        
        # Overplot spectrum smoothed using a 1-D Gaussian Kernel in black
        plt.plot(w, convolve(f, Gaussian1DKernel(2)), color = 'k', label = 'Smoothed spectrum')
    
        plt.plot(w, iv, color = 'orange', alpha = 0.5, label = 'noise')
    
        plt.legend()
        plt.savefig(str(path) + f"/{targetid}.png", bbox_inches = 'tight')

In [ ]:
plot_spec(spectra = spectra, end = len(spectra), init = 0, path = "/pscratch/sd/j/juikuan/DESI_LAE_spectrum/OII_catalog", Lya = False)

In [ ]:
def fitting_Lya(spectra, end, result_path = None, spectra_path = None, init = 0, file_name = "OII_fitting"):
    
    LAE = []
    NLAE_high = []
    NLAE_medium = []
    NLAE_low = []
    
    if os.path.isdir(str(result_path)):
        pass
    else:
        os.mkdir(str(result_path))

    if os.path.isdir(str(spectra_path)):
        pass
    else:
        os.mkdir(str(spectra_path))

    TARGETID = []
    CHI2_MIN = []
    REDUCED_CHI2 = []
    PARAMS = []
    ASYMPTOTIC_ERROR = []
    STANDARD_ERROR = []
    WRONG_Z = []
    TRUE_Z = []
    LYA_FLUX = []
    LYA_EW = []
    n = 0
    
    for i in range(init, end):
        n += 1
        print(f"\rProcess: {str(n / end * 100)} %", end = "")
        record = spectra[i]
    
        flux = record.flux
        wavelength = record.wavelength
        model = record.model
        redshift = record.redshift
        ivar = record.ivar
        targetid = record.specid
        sparcl_id = record.sparcl_id
    
        min = 3726.032 * (redshift + 1) - 25
        max = 3726.032 * (redshift + 1) + 30
        mask_min = wavelength > min
        mask_max = wavelength < max
        w = wavelength[mask_min & mask_max]
        f = flux[mask_min & mask_max]
        m = model[mask_min & mask_max]
        iv = np.sqrt(ivar[mask_min & mask_max]) ** -1
        
        if redshift > 0.4:
            continue
        if np.max(f) == np.inf:
            f[np.where(f == np.inf)] = np.median(f)
        elif np.max(m) == np.inf:
            m[np.where(m == np.inf)] = np.median(fm)
        elif np.max(iv) == np.inf:
            iv[np.where(iv == np.inf)] = np.median(iv)

        def my_model(p, x):
            A, mu, sigma, zerolev = p
            return( A * np.exp(-(x-mu)*(x-mu)/(2.0*sigma*sigma)) + zerolev )
            
        def my_residuals(p, data):
            A, mu, sigma, zerolev = p
            x, y, err = data
            return (y - my_model(p,x)) / err

        fittings = list()
        for j in range(math.ceil(3726.032 * (redshift + 1)) - 10, math.floor(3726.032 * (redshift + 1)) + 15):

            fitobj = kmpfit.Fitter(residuals = my_residuals, data = (w, f, iv), parinfo = [{"limits": [0, 100000000]}, {"limits": [3726.032 * (redshift + 1) - 10, 3726.032 * (redshift + 1) + 15]}, {"limits": [0, 10000000]}, None])
            p0 = [np.amax(f), j, 2.5, 0.1]
            
            fitobj.fit(params0 = p0)

            chi2_min = fitobj.chi2_min
            reduced_chi2 = fitobj.rchi2_min
            params = fitobj.params
            asymptotic_error = fitobj.xerror
            standard_errors = fitobj.stderr
            
            wrong_z = redshift
            true_z = fitobj.params[1] / 1215.67 - 1
            
            area = fitobj.params[2] * fitobj.params[0] * np.sqrt(2 * math.pi)
            EW = area / (fitobj.params[3] * (fitobj.params[1] / 1215.67))

            fittings.append({"targetid": targetid, "chi2_min": chi2_min, "reduced_chi2": reduced_chi2, "params": params, "asymptotic_error": list(asymptotic_error), "standard_errors": list(standard_errors),"wrong_z": wrong_z, "true_z": true_z, "EW": EW, "area": area})
            fittings.sort(key = lambda x: (x["reduced_chi2"] - 1) ** 2)

        TARGETID.append(fittings[0]['targetid'])
        CHI2_MIN.append(fittings[0]['chi2_min'])
        REDUCED_CHI2.append(fittings[0]['reduced_chi2'])
        PARAMS.append(fittings[0]['params'])
        ASYMPTOTIC_ERROR.append(fittings[0]['asymptotic_error'])
        STANDARD_ERROR.append(fittings[0]['standard_errors'])
        WRONG_Z.append(fittings[0]['wrong_z'])
        TRUE_Z.append(fittings[0]['true_z'])
        LYA_FLUX.append(fittings[0]['area'])
        LYA_EW.append(fittings[0]['EW'])
        
        if spectra_path != None:
            plt.close()
            rc('font', size = 9)
            rc('legend', fontsize = 8)
            
            plt.title(f"Target ID = {fittings[0]['targetid']}\n"
                      f"chi^2 min = {fittings[0]['chi2_min']}\n"
                      f"Reduced Chi^2 = {fittings[0]['reduced_chi2']}\n"
                      f"Redshift = {fittings[0]['true_z']}\n"
                      f"Params = {fittings[0]['params']}\n"
                      f"Flux = {fittings[0]['area']}\n"
                      f"Rest Frame EW = {fittings[0]['EW']}\n"
                      f"Standard errors = {fittings[0]['standard_errors']}\n"
                      f"Asymptotic error = {fittings[0]['asymptotic_error']}", loc = 'left')
            
            plt.xlabel('$\lambda\ [\AA]$')
            plt.ylabel('$f_{\lambda}$ $(10^{-17}$ $erg$ $s^{-1}$ $cm^{-2}$ $\AA^{-1})$')
            plt.plot(w, f, color='k', alpha = 0.4, label = 'Unsmoothed spectrum')
            plt.plot(w, iv, color='g', alpha = 0.4, label = 'noise')
            plt.plot(w, my_model(fittings[0]['params'], w), 'b', label = "Fit with kmpfit")
            plt.legend()
            plt.savefig(str(spectra_path) + f"{targetid}_OII.png", bbox_inches ='tight')
        
    Lya_emission = []
    for i in (TARGETID, CHI2_MIN, REDUCED_CHI2, PARAMS, ASYMPTOTIC_ERROR, STANDARD_ERROR, WRONG_Z, TRUE_Z, LYA_FLUX, LYA_EW):
        Lya_emission.append(i)

    if result_path != None:
        t = Table(Lya_emission, names = ("TARGETID", "CHI2_MIN", "REDUCED_CHI2", "PARAMS", "ASYMPTOTIC_ERROR", "STANDARD_ERROR", "WRONG_Z", "TRUE_Z", "LYA_FLUX", "LYA_EW"))
        t.write(str(result_path) + str(file_name) + ".fits", format = "fits")
    else:
        pass

In [ ]:
spectra, results, records = selection_OII(path = "../final_dataset/")

In [ ]:
fitting_Lya(spectra = spectra, result_path = "../final_dataset/", spectra_path = "../final_dataset/OII_spectra/", end = len(spectra))

In [ ]:
plot_spec(spectra = spectra, end = len(spectra), init = 0, path = "../final_dataset/OII_spectra_whole/", Lya = False)